In [1]:
%matplotlib notebook
import pandas as pd
import geopandas as gpd
import glob
import os
import nivapy
from shapely.geometry import Point, Polygon

In [2]:
# Connect to RESA
ora_eng = nivapy.da.connect(src='nivabase')

Username: ········
Password: ········
Connection successful.


In [3]:
# Connect to spatial db
pg_eng = nivapy.da.connect(src='postgres')

Username: ········
Password: ········
Connection successful.


# Critical Loads: new grid

The latest NILU deposition data for the Critical Loads work uses a finer resolution grid than has been supplied previously. For most of the work so far we have continued to use the coarser BLR grid, but Kari would like to switch to the higher resolution grid in the future. This notebook explores the new data and, especially, the new 0.1 degree grid.

## 1. Read high resolution data

In [4]:
# Read NILU data
data_fold = r'C:\Data\James_Work\Staff\Kari_A\Critical_Loads\raw_data'
search_path = os.path.join(data_fold, '*.dat')
file_list = glob.glob(search_path)

df_list = []
for fpath in file_list:
    # Get par name
    name = os.path.split(fpath)[1].split('_')[:2]
    name = '_'.join(name)
    
    # Read file
    df = pd.read_csv(fpath, delim_whitespace=True, header=None,
                     names=['lat', 'lon', name])
    df.set_index(['lat', 'lon'], inplace=True)    
    df_list.append(df)

# Combine
df = pd.concat(df_list, axis=1)
df.reset_index(inplace=True)

df.head()

,lat,lon,tot_nhx,tot_nox,tot_n,tot_s
0,50.05,3.05,270.87,226.36,497.23,118.59
1,50.05,3.15,240.50,214.75,455.25,114.76
2,50.05,3.25,259.19,214.43,473.62,122.13
3,50.05,3.35,252.06,219.67,471.73,108.88
4,50.05,3.45,332.82,241.87,574.69,126.64


## 2. Grid definition

The original BLR grid has a resolution of 0.125 degrees in the N-S direction and 0.25 degrees in the E-W direction. One option is therefore to calculate a new grid from the 0.1 degree data and then use this in exactly the same way as the BLR grid (after calculating the cell areas using any "equal area" projection). The advantage of this approach is that it can be substituted easily into the existing workflow. The downside is that using an irregular vector dataset to represent a grid is inefficient: a raster in an appropriate projected co-ordinate system would be better.

To save time, it is probably easier to continue with the vector workflow for this year, but it's worth **discussing the raster option with Kari**.

### 2.1. Build vector grid

In [5]:
def build_poly(row, res=0.1):
    """ Builds a vector grid of polygons with a resolution of "res"
        decimal degrees from a set of points specifying cell centres.
    """
    from shapely.geometry import Polygon
    
    # Get shift = half of res
    sft = res / 2.
    
    # Get centre co-ords
    x, y = row['lon'], row['lat']
    
    # Build polygon
    cords = [(x-sft, y-sft), (x-sft, y+sft), 
             (x+sft, y+sft), (x+sft, y-sft),
             (x-sft, y-sft)]
    poly = Polygon(cords)
    
    return poly

# Build vector grid
df['geometry'] = df.apply(build_poly, axis=1)

# Convert to geo df
crs = {'init': 'epsg:4326'}
gdf = gpd.GeoDataFrame(df, crs=crs, geometry='geometry')

# Save shp
out_shp = (r'C:\Data\James_Work\Staff\Kari_A\Critical_Loads\GIS'
           r'\Shapefiles\crit_lds_0_1deg_grid.shp')
gdf.to_file(driver='ESRI Shapefile', filename=out_shp)

gdf.head()

,lat,lon,tot_nhx,tot_nox,tot_n,tot_s,geometry
0,50.05,3.05,270.87,226.36,497.23,118.59,"POLYGON ((3 50, 3 50.09999999999999, 3.1 50.09..."
1,50.05,3.15,240.50,214.75,455.25,114.76,"POLYGON ((3.1 50, 3.1 50.09999999999999, 3.2 5..."
2,50.05,3.25,259.19,214.43,473.62,122.13,"POLYGON ((3.2 50, 3.2 50.09999999999999, 3.3 5..."
3,50.05,3.35,252.06,219.67,471.73,108.88,"POLYGON ((3.3 50, 3.3 50.09999999999999, 3.4 5..."
4,50.05,3.45,332.82,241.87,574.69,126.64,"POLYGON ((3.4 50, 3.4 50.09999999999999, 3.5 5..."


### 2.2. Clip to coastline

The BLR grid is clipped to the Norwegian coastline, so many of the cells are actually complex polygons. The new deposition dataset extends well beyond Norway, so to keep data sizes manageable (and to stay close to the previous methodology), it is necessary to "clip" the new dataset in a similar way.

This is most easily done using `Dissolve` followed by `Clip` in ArcGIS. Since the operation only needs to be done once, it does not need to be scripted. The output file from this is *crit_lds_0_1deg_grid_clip.shp*.

### 2.3. Calculate areas

It is not possible to get accurate area calculations in a geographic (lat/lon) co-ordinate system. Instead, I will temporarily re-project the data to an equal area projection, calculate the cell areas, and then join the results back to the original data in WGS84 geographic co-ordinates.

I will also create a unique ID for each cell, by combining the lat and lon co-ordinates (multipled by 100 and padded to 4 digits).

In [6]:
# Read clipped data
shp_path = (r'C:\Data\James_Work\Staff\Kari_A\Critical_Loads\GIS'
            r'\Shapefiles\crit_lds_0_1deg_grid_clip.shp')
gdf = gpd.read_file(shp_path)

# Calculate unique integer cell ID as latlon 
# (both *100 and padded to 4 digits)
gdf['cell_id'] = ((gdf['lat']*100).astype(int).map('{:04d}'.format) + 
                  (gdf['lon']*100).astype(int).map('{:04d}'.format))
gdf['cell_id'] = gdf['cell_id'].astype(int)
assert gdf['cell_id'].is_unique

# Project to Equal Area Cylindrical
gdf2 = gdf.to_crs({'proj':'cea'})

# Calc area in m2
gdf2['area_m2'] = gdf2['geometry'].area

# Add areas to original GDF
gdf['area_m2'] = gdf2['area_m2']

# Save
gdf.to_file(driver='ESRI Shapefile', filename=shp_path)

gdf.head()

,lat,lon,cell_id,area_m2,geometry
0,62.05,6.75,62050675,5.830022e+07,"POLYGON ((6.799999999999995 61.99999999999997,..."
1,67.95,16.55,67951655,4.507216e+07,"POLYGON ((16.59999999999998 68.00000000000003,..."
2,60.35,5.95,60350595,6.151349e+07,"POLYGON ((5.999999999999995 60.30000000000003,..."
3,59.35,6.35,59350635,5.808014e+07,(POLYGON ((6.400000000000039 59.30404746914273...
4,69.85,28.55,69842855,4.287583e+07,"POLYGON ((28.59999999999999 69.8, 28.509245922..."


## 3. Add to databases

This geodataframe is the new equivalent of the old BLR grid. It can therefore be stored in the RESA2 database in the same way. As this is a spatial dataset, it also seems sensible to store it in the new spatial database.

### 3.1. Add to PostGIS

In [7]:
# Reorder cols
gdf = gdf[['geometry', 'cell_id', 'lat', 'lon', 'area_m2']]

# Write to db
nivapy.da.gdf_to_postgis(gdf, 'dep_grid_0_1deg', pg_eng, 
                         'public_dep_grid_0_1deg',
                         schema='public', if_exists='replace',
                         index=False)

WARNING! The dataframe contains mixed geometries:
  set(['Polygon', 'MultiPolygon'])
These will be cast to "Multi" type. If this is not what you want, consider using gdf.explode() first


C:\Data\Anaconda2\lib\site-packages\sqlalchemy\dialects\postgresql\base.py:2543: SAWarning: Did not recognize type 'geometry' of column 'geom'
  (attype, name))


### 3.2. Add to RESA2

In [8]:
# Remove spatial info
df = pd.DataFrame(gdf)
del gdf['geometry']

# Write to db
df.to_sql('dep_grid_0_1deg', ora_eng, schema='resa2', 
          if_exists='replace', index=False)